In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum as sum_, first, round, min, when, lit, to_date, to_timestamp, date_format
from pyspark.sql.types import DoubleType, IntegerType, StringType, DateType, TimestampType

# Configurações do Spark
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.11.901')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')

# Criar sessão Spark
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# # Ler o CSV, já tratando "NULL" como valor nulo
# TabelaCompleta = spark.read.option('delimiter', ',') \
#               .option('header', 'true') \
#               .option('nullValue', 'null') \
#               .csv('s3a://bucket-raw-upa-connect-1/tabela_historico_sensor1.csv')

# Lista de arquivos CSV a serem lidos
csv_files = [
    'dados_2025_06_02.csv',
    'dados_2025_05_27.csv',
    'dados_2025_05_28.csv',
    'dados_2025_05_29.csv',
    'dados_2025_05_30.csv',
    'dados_2025_05_31.csv',
    'dados_2025_06_01.csv',
    'dados_2025_05_20.csv',
    'dados_2025_05_21.csv',
    'dados_2025_05_22.csv',
    'dados_2025_05_23.csv',
    'dados_2025_05_24.csv',
    'dados_2025_05_25.csv',
    'dados_2025_05_26.csv'
]

# Prefixo do caminho S3
s3_prefix = 's3a://bucket-raw-upa-connect-1/'

# Criar a lista completa de caminhos S3
s3_paths = [s3_prefix + file for file in csv_files]

# Ler os CSVs necessários a partir da lista de caminhos
TabelaCompleta = spark.read.option('delimiter', ',') \
                             .option('header', 'true') \
                             .option('nullValue', 'null') \
                             .csv(s3_paths) # Agora lê múltiplos arquivos


# Ler o CSV, já tratando "NULL" como valor nulo
upa_df = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .csv('s3a://bucket-trusted-upa-connect-1/upa.csv')

paciente_df = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .csv('s3a://bucket-trusted-upa-connect-1/paciente.csv')

sensor_df = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .csv('s3a://bucket-trusted-upa-connect-1/sensor.csv')


unidadeDeMedida_df = spark.read.option('delimiter', ',') \
              .option('header', 'true') \
              .csv('s3a://bucket-trusted-upa-connect-1/UnidadeDeMedida.csv')

# Converter 'valor' para DoubleType e 'fk_upa' para IntegerType
# a = a.withColumn('valor', col('valor').cast(DoubleType())) \
#      .withColumn('fk_paciente', col('fk_paciente').cast(IntegerType()))

TabelaCompleta.show()
# Conta as linhas
num_linhas = TabelaCompleta.count()

# Imprime o resultado
print(f"Número de linhas: {num_linhas}")
# upa_df.show()
# paciente_df.show(170)
# sensor_df.show()
# unidadeDeMedida_df.show()




:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-957192c3-2172-46bd-bec5-156d80c75085;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 528ms :: artifacts dl 17ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.12.262] in [default]
	---------------------------------------------------------------------
	|     

+-------------------+--------------------+---------+--------------+-----------+------+
|          data_hora|               valor|fk_sensor|fk_unid_medida|fk_paciente|fk_upa|
+-------------------+--------------------+---------+--------------+-----------+------+
|2025-06-01T00:05:00|                  23|        1|          null|       null|     1|
|2025-06-01T00:10:00|                  21|        1|          null|       null|     1|
|2025-06-01T00:15:00|                  19|        1|          null|       null|     1|
|2025-06-01T00:20:00|                  18|        1|          null|       null|     1|
|2025-06-01T00:25:00|                  15|        1|          null|       null|     1|
|2025-06-01T00:30:00|                null|        1|          null|       null|     1|
|2025-06-01T00:35:00|                  11|        1|          null|       null|     1|
|2025-06-01T00:40:00|                  11|        1|          null|       null|     1|
|2025-06-01T00:45:00|                   9| 

Número de linhas: 2193394


In [2]:
# 1. Filtrar pelos sensores desejados e remover valores nulos em 'valor'
df_filtrado_inicial = TabelaCompleta.filter(
    (col("fk_sensor") == 3) | (col("fk_sensor") == 4)
).filter(
    col('valor').isNotNull()
)

# 2. Aplicar filtros de outlier específicos para cada sensor
condicao_sensor_3 = (col('fk_sensor') == 3) & (col('valor') >= 0) & (col('valor') <= 100)
condicao_sensor_4 = (col('fk_sensor') == 4) & (col('valor') >= 34) & (col('valor') <= 42)

df_filtrado_final = df_filtrado_inicial.filter(
    condicao_sensor_3 | condicao_sensor_4
).orderBy("fk_paciente")

df_filtrado_final = df_filtrado_final.withColumn('valor', col('valor').cast(DoubleType())) \
     .withColumn('fk_paciente', col('fk_paciente').cast(IntegerType()))

# df_filtrado_final.show()

# --- ALTERAÇÃO PRINCIPAL AQUI: Agrupar por paciente E por dia ---
df_resultado_final = df_filtrado_final.groupBy(
    'fk_paciente',
    to_date(col('data_hora')).alias('data_agrupamento_temporaria') # Coluna temporária para agrupar por dia
).agg(
    # Pega a menor data_hora para cada paciente por dia (pode ser ajustado para 'first' se preferir)
    min(col('data_hora')).alias('data_hora'),
    # Pega o primeiro fk_upa encontrado para cada paciente por dia (assumindo que fk_upa é o mesmo para o paciente)
    min(col('fk_upa')).alias('fk_upa'),
    # Média da Oximetria (fk_sensor = 3)
    round(avg(when(col('fk_sensor') == 3, col('valor'))), 1).alias('Oximetria_Media'),
    # Média da Temperatura (fk_sensor = 4)
    round(avg(when(col('fk_sensor') == 4, col('valor'))), 1).alias('Temperatura_Media')
).orderBy('fk_paciente', 'data_agrupamento_temporaria') # Ordena para facilitar a visualização por paciente e dia

# df_resultado_final.show(120)

# --- Adicionando as Colunas "Legenda Gravidade" e "Cor Indicativa" ---

df_com_legenda = df_resultado_final.withColumn('Legenda Gravidade',
    when(col('Oximetria_Media') < 87, 'Emergência - Oxigenação Crítica') # Mais grave, prioridade máxima
    .when(col('Temperatura_Media') > 39.5, 'Emergência - Febre Hipertermia') # Muito grave
    .when((col('Temperatura_Media') < 35.0) & (col('Oximetria_Media') < 90), 'Alerta - Hipotermia e Hipoxemia') # Combinação perigosa
    .when((col('Temperatura_Media') > 38.5) & (col('Oximetria_Media') <= 92), 'Alerta - Febre Alta e Oxigenação Reduzida') # Grave
    .when(col('Temperatura_Media') > 38.0, 'Atenção - Febre') # Atenção
    .when((col('Oximetria_Media') >= 87) & (col('Oximetria_Media') <= 92), 'Atenção - Oxigenação Baixa') # Atenção
    .when(col('Temperatura_Media') < 35.5, 'Atenção - Temperatura Baixa') # Atenção
    .otherwise('Normal - Sem Alertas') # Normal
)

# Mapeamento de prioridade numérica para a Cor Indicativa (cores ajustadas para gravidade)
df_resultado_final = df_com_legenda.withColumn('Cor Indicativa',
    when(col('Legenda Gravidade') == 'Emergência - Oxigenação Crítica', '#8B0000') # Vermelho Escuro: Perigo Imediato
    .when(col('Legenda Gravidade') == 'Emergência - Febre Hipertermia', '#DC143C') # Carmesim: Muito Crítico
    .when(col('Legenda Gravidade') == 'Alerta - Hipotermia e Hipoxemia', '#FF4500') # Laranja Avermelhado: Urgência Elevada
    .when(col('Legenda Gravidade') == 'Alerta - Febre Alta e Oxigenação Reduzida', '#FF8C00') # Laranja Escuro: Alerta Sério
    .when(col('Legenda Gravidade') == 'Atenção - Febre', '#FFD700') # Dourado: Atenção Moderada
    .when(col('Legenda Gravidade') == 'Atenção - Oxigenação Baixa', '#32CD32') # Verde Limão: Atenção Leve (mas ainda requer observação)
    .when(col('Legenda Gravidade') == 'Atenção - Temperatura Baixa', '#4169E1') # Azul Real: Atenção Leve
    .otherwise('#008000') # Verde Escuro: Tudo Certo
)

## Juntando Dados e Reestruturando Colunas

### 1. Renomear `fk_paciente` para o Nome do Paciente

df_final = df_resultado_final.join(
    paciente_df,
    df_resultado_final.fk_paciente == paciente_df.id_paciente,
    "left"
).withColumnRenamed("nome", "nome_paciente") \
 .drop("fk_paciente") \
 .drop("id_paciente") \
 .drop("cpf") \
 .drop("data_nascimento") \
 .drop("biometria")

# --- AJUSTE AQUI: Renomear e reestruturar colunas de data/hora ---
# A coluna 'data_hora' agora representa a primeira medição do dia para aquele paciente.
# A coluna 'data_agrupamento_temporaria' pode ser descartada ou renomeada, mas para manter a consistência,
# vamos usar 'data_hora' para derivar 'data_chegada' e 'horario_chegada'.
df_final = df_final.withColumn('data_chegada', to_date(col('data_hora'), "yyyy-MM-dd'T'HH:mm:ss").cast(DateType())) \
                     .withColumn('horario_chegada', date_format(col('data_hora'), "HH:mm:ss").cast(StringType())) \
                     .drop('data_hora') \
                     .drop('data_agrupamento_temporaria') # Remove a coluna temporária usada para o agrupamento

# `join` com o `upa_df` e depois excluindo colunas desnecessarias.
df_final = df_final.join(
    upa_df,
    df_final.fk_upa == upa_df.id_upa,
    "left"
).withColumnRenamed("nome", "nome_da_upa") \
 .drop("id_upa") \
 .drop("cnpj") \
 .drop("telefone") \
 .drop("capacidade_atendimento") \
 .drop("fk_endereco")

## Definir a ordem final das colunas

colunas_finais_ordenadas = [
    'nome_paciente',
    'Oximetria_Media',
    'Temperatura_Media',
    'Legenda Gravidade',
    'Cor Indicativa',
    'data_chegada',
    'horario_chegada',
    'fk_upa',
    'nome_da_upa'
]


df_final = df_final.select(colunas_finais_ordenadas)

# Mostrar o DataFrame resultante
df_final.show(120)


+--------------+---------------+-----------------+--------------------+--------------+------------+---------------+------+--------------------+
| nome_paciente|Oximetria_Media|Temperatura_Media|   Legenda Gravidade|Cor Indicativa|data_chegada|horario_chegada|fk_upa|         nome_da_upa|
+--------------+---------------+-----------------+--------------------+--------------+------------+---------------+------+--------------------+
|     Ana Souza|           89.8|             38.2|     Atenção - Febre|       #FFD700|  2025-05-20|       00:30:00|     1|     UPA 21 DE JUNHO|
|     Ana Souza|           91.8|             38.2|     Atenção - Febre|       #FFD700|  2025-05-21|       00:55:00|     1|     UPA 21 DE JUNHO|
|     Ana Souza|           88.5|             38.0|Atenção - Oxigena...|       #32CD32|  2025-05-22|       00:00:00|     1|     UPA 21 DE JUNHO|
|     Ana Souza|           88.9|             38.1|     Atenção - Febre|       #FFD700|  2025-05-23|       02:00:00|     1|     UPA 21 DE

In [3]:
df_final.coalesce(1) \
        .write \
        .option('header', 'true') \
        .mode('overwrite') \
        .csv('s3a://bucket-trusted-upa-connect-1/grafico_oximetria_temperatura.csv')

25/06/15 00:18:31 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/06/15 00:18:31 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [4]:
# --- Instalação de bibliotecas (adicione esta seção no início do seu script) ---
import subprocess
import sys

# Função para instalar pacotes
def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"'{package}' instalado com sucesso.")
    except Exception as e:
        print(f"Erro ao instalar '{package}': {e}")
        # Em ambientes de produção, você pode querer adicionar um exit(1) aqui
        # para parar a execução se uma dependência crítica não puder ser instalada.

# Instalar as bibliotecas necessárias
install_package("pandas")
install_package("gspread")
install_package("google-auth-oauthlib") # Necessário para google.oauth2.service_account.Credentials
install_package("boto3") # Para interagir com o S3
# --- Fim da seção de instalação de bibliotecas ---


# --- CÓDIGO PARA ENVIAR PARA O GOOGLE SHEETS ---
import pandas as pd # <-- MOVIDO PARA AQUI PARA GARANTIR QUE 'pd' ESTEJA NO ESCOPO
import gspread
from google.oauth2.service_account import Credentials
import boto3
import os
import sys

# === CONFIGURAÇÕES PARA O GOOGLE SHEETS ===
GOOGLE_SHEET_ID = '1i6BfuZXPOcTp6BFAiVkpktOBym0HtakZacUKfDzu1zI'
S3_BUCKET_CREDENTIALS = 'bucket-trusted-upa-connect-1'
S3_KEY_CREDENTIALS = 'credenciais.json'
LOCAL_CREDENTIALS_PATH = '/tmp/credenciais.json'

# === AUTENTICAÇÃO GOOGLE ===
print(f"Baixando credenciais do S3: s3://{S3_BUCKET_CREDENTIALS}/{S3_KEY_CREDENTIALS} para {LOCAL_CREDENTIALS_PATH}...")
s3 = boto3.client('s3')
try:
    s3.download_file(S3_BUCKET_CREDENTIALS, S3_KEY_CREDENTIALS, LOCAL_CREDENTIALS_PATH)
    print("Credenciais baixadas com sucesso.")
except Exception as e:
    print(f"Erro ao baixar credenciais do S3: {e}")
    sys.exit(1)

scopes = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
credenciais = Credentials.from_service_account_file(LOCAL_CREDENTIALS_PATH, scopes=scopes)
cliente = gspread.authorize(credenciais)
planilha = cliente.open_by_key(GOOGLE_SHEET_ID)

# === ENVIA O DATAFRAME FINAL PARA A NOVA ABA "teste" ===
NOME_ABA_OXIMETRIA_PACIENTE = "OximetriaPaciente"
print(f"Enviando DataFrame final para a aba '{NOME_ABA_OXIMETRIA_PACIENTE}'...")

# Converter Spark DataFrame para Pandas DataFrame
try:
    df_pandas = df_final.toPandas()
    print("DataFrame Spark convertido para Pandas com sucesso.")
except Exception as e:
    print(f"Erro ao converter Spark DataFrame para Pandas: {e}")
    print("O DataFrame pode ser muito grande para a memória do driver ou a sessão Spark está inválida.")
    sys.exit(1)

# --- NOVO TRECHO: Ajustar tipagem das colunas para o Google Sheets ---
# Apenas formata colunas de data/hora para string ISO 8601.
# As demais colunas (numéricas, strings, etc.) são mantidas em seus tipos nativos no Pandas.
# O gspread e o Google Sheets farão a inferência de tipo para elas.

# Coluna 'data_chegada': assegurar que é string DD/MM/YYYY
if 'data_chegada' in df_pandas.columns:
    if pd.api.types.is_datetime64_any_dtype(df_pandas['data_chegada']):
        df_pandas['data_chegada'] = df_pandas['data_chegada'].dt.strftime('%d/%m/%Y') # <-- MUDANÇA AQUI
    elif pd.api.types.is_string_dtype(df_pandas['data_chegada']):
        # Tentar converter strings que podem não estar em formato padrão para datetime,
        # e depois formatar para DD/MM/YYYY. 'errors=coerce' transforma não-datas em NaT.
        df_pandas['data_chegada'] = pd.to_datetime(df_pandas['data_chegada'], errors='coerce').dt.strftime('%d/%m/%Y') # <-- MUDANÇA AQUI
    # Preencher valores nulos (NaT) com string vazia, pois None também pode causar problemas.
    df_pandas['data_chegada'] = df_pandas['data_chegada'].fillna('')

# Coluna 'horario_chegada': assegurar que é string HH:MM:SS
if 'horario_chegada' in df_pandas.columns:
    if pd.api.types.is_datetime64_any_dtype(df_pandas['horario_chegada']):
        df_pandas['horario_chegada'] = df_pandas['horario_chegada'].dt.strftime('%H:%M:%S')
    elif pd.api.types.is_string_dtype(df_pandas['horario_chegada']):
        # Se já é string (como no seu caso vindo do Spark), apenas garanta que nulos sejam preenchidos.
        df_pandas['horario_chegada'] = df_pandas['horario_chegada'].fillna('')


# Nenhuma conversão global df_pandas.astype(str) é necessária aqui.
# gspread é capaz de lidar com int, float, bool, e strings diretamente.

dados = [df_pandas.columns.tolist()] + df_pandas.values.tolist()

try:
    aba = planilha.worksheet(NOME_ABA_OXIMETRIA_PACIENTE)
    aba.clear()
    print(f"Aba '{NOME_ABA_OXIMETRIA_PACIENTE}' encontrada e limpa.")
except gspread.exceptions.WorksheetNotFound:
    aba = planilha.add_worksheet(title=NOME_ABA_OXIMETRIA_PACIENTE, rows=str(len(dados) + 100), cols=str(len(df_pandas.columns) + 10))
    print(f"Aba '{NOME_ABA_OXIMETRIA_PACIENTE}' criada.")

try:
    aba.update(dados)
    print(f"Dados do DataFrame final enviados com sucesso para a aba '{NOME_ABA_OXIMETRIA_PACIENTE}'.")
except Exception as e:
    print(f"Erro ao atualizar a aba do Google Sheets: {e}")

# Opcional: Remover o arquivo de credenciais temporário
if os.path.exists(LOCAL_CREDENTIALS_PATH):
    os.remove(LOCAL_CREDENTIALS_PATH)
    print(f"Arquivo de credenciais temporário '{LOCAL_CREDENTIALS_PATH}' removido.")

# Parar a sessão Spark
# spark.stop()

'pandas' instalado com sucesso.


'gspread' instalado com sucesso.


'google-auth-oauthlib' instalado com sucesso.


'boto3' instalado com sucesso.
Baixando credenciais do S3: s3://bucket-trusted-upa-connect-1/credenciais.json para /tmp/credenciais.json...


/usr/local/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


Credenciais baixadas com sucesso.
Enviando DataFrame final para a aba 'OximetriaPaciente'...


DataFrame Spark convertido para Pandas com sucesso.
Aba 'OximetriaPaciente' encontrada e limpa.
Dados do DataFrame final enviados com sucesso para a aba 'OximetriaPaciente'.
Arquivo de credenciais temporário '/tmp/credenciais.json' removido.
